In [2]:
import numpy as np

def generate_episode():
    S = []
    A = []
    R = []
    T_max = 1000

    # Random initial state
    S.append([np.random.randint(4), np.random.randint(4)])

    if S[-1]!=[0,0] and S[-1]!=[3,3]:
        A.append(np.random.randint(4))
        R.append(-1)
    else:
        A.append(0)
        R.append(0)
    
    for t in range(T_max):
        if S[-1] == [0,0] or S[-1]==[3,3]:
            break
        elif A[-1] == 0:
            next_state = [max(0, S[-1][0]-1), S[-1][1]]
        elif A[-1] == 1:
            next_state = [S[-1][0], min(3, S[-1][1]+1)]
        elif A[-1] == 2:
            next_state = [min(3, S[-1][0]+1), S[-1][1]]
        elif A[-1] == 3:
            next_state = [S[-1][0], max(0, S[-1][1]-1)]
        
        S.append(next_state)

        if S[-1]!=[0,0] and S[-1]!=[3,3]:
            A.append(np.random.randint(4))
            R.append(-1)
        else:
            A.append(0)
            R.append(0)
    
    return S, A, R


In [3]:
Q = np.zeros((4,4,4))
C = np.zeros((4,4,4))
policy = np.zeros((4,4), dtype=int)

# implement off-policy MC control

for n_episodes in range(1000):
    S, A, R = generate_episode()
    G = 0
    W = 1

    for t in range(len(S)-2, -1, -1):
        G = 1*G + R[t]
        C[S[t][0], S[t][1], A[t]] += W
        Q[S[t][0], S[t][1], A[t]] = Q[S[t][0], S[t][1], A[t]] + (W/C[S[t][0], S[t][1], A[t]]) * (G-Q[S[t][0], S[t][1], A[t]])
        policy[S[t][0], S[t][1]] = int(np.argmax(Q[S[t][0], S[t][1]]))
        if policy[S[t][0], S[t][1]] != A[t]:
            break
        W = W*4

print(policy)

# for n_episodes in range(1000000):
#     S, A, R = generate_episode()
#     # print(len(S))
#     G = 0
#     W = 1

#     for t in range(len(S)-2, -1, -1):
#         G = 1*G + R[t]
#         C[S[t][0], S[t][1], A[t]] += W
#         Q[S[t][0], S[t][1], A[t]] = Q[S[t][0], S[t][1], A[t]] + (W/C[S[t][0], S[t][1], A[t]]) * (G-Q[S[t][0], S[t][1], A[t]])
#         # print(Q[2, 2])
#         policy[S[t][0], S[t][1]] = int(np.argmax(Q[S[t][0], S[t][1]]))
#         if policy[S[t][0], S[t][1]] != A[t]:
#             break
#         W = W*4

# print(policy)
    

[[0 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
